In [1]:
Boyle.mk("env")
Boyle.activate("env")
Boyle.install({:emel, "~> 0.3.0"})
Boyle.install({:csv, "~> 2.0.0"})
alias Emel.Ml.DecisionTree, as: DT
alias Emel.Help.Model, as: Mdl

All dependencies up to date
All dependencies up to date
Resolving Hex dependencies...
Dependency resolution completed:
Unchanged:
  csv 2.0.0
  emel 0.3.0
  parallel_stream 1.0.6
All dependencies up to date
Resolving Hex dependencies...
Dependency resolution completed:
Unchanged:
  csv 2.0.0
  emel 0.3.0
  parallel_stream 1.0.6
All dependencies up to date


  mix.exs:22

  mix.exs:1

  mix.exs:22

  mix.exs:1

  /home/dimos/projects/elixir/IElixir/envs/env/deps/parallel_stream/mix.exs:17

  /home/dimos/projects/elixir/IElixir/envs/env/deps/csv/mix.exs:18

  mix.exs:22

  mix.exs:1

  /home/dimos/projects/elixir/IElixir/envs/env/deps/parallel_stream/mix.exs:17

  /home/dimos/projects/elixir/IElixir/envs/env/deps/csv/mix.exs:18



Emel.Help.Model

In [2]:
dataset = "../emel/notebooks/datasets/StudentsPerformance.csv"
    |> File.stream!()
    |> CSV.decode!(headers: true)
    |> Enum.to_list()
hd(dataset)

%{"gender" => "female", "lunch" => "standard", "math score" => "72", "parental level of education" => "bachelor's degree", "race/ethnicity" => "group B", "reading score" => "72", "test preparation course" => "none", "writing score" => "74"}

In [3]:
categorizer =  Mdl.categorizer(["Fail", 40, "Sub-Pass", 45, "Pass"])

#Function<3.4184123/1 in Emel.Help.Model.categorizer/1>

In [4]:
parsed_dataset = Enum.map(
  dataset,
  fn %{"writing score" => score} = row ->
    grade = score
    |> Mdl.parse()
    |> categorizer.()
    row
    |> Map.drop(["math score", "reading score", "writing score"])
    |> Map.put("writing grade", grade)
  end
)


[%{"gender" => "female", "lunch" => "standard", "parental level of education" => "bachelor's degree", "race/ethnicity" => "group B", "test preparation course" => "none", "writing grade" => "Pass"}, %{"gender" => "female", "lunch" => "standard", "parental level of education" => "some college", "race/ethnicity" => "group C", "test preparation course" => "completed", "writing grade" => "Pass"}, %{"gender" => "female", "lunch" => "standard", "parental level of education" => "master's degree", "race/ethnicity" => "group B", "test preparation course" => "none", "writing grade" => "Pass"}, %{"gender" => "male", "lunch" => "free/reduced", "parental level of education" => "associate's degree", "race/ethnicity" => "group A", "test preparation course" => "none", "writing grade" => "Sub-Pass"}, %{"gender" => "male", "lunch" => "standard", "parental level of education" => "some college", "race/ethnicity" => "group C", "test preparation course" => "none", "writing grade" => "Pass"}, %{"gender" => "f

In [5]:
hd(parsed_dataset)

%{"gender" => "female", "lunch" => "standard", "parental level of education" => "bachelor's degree", "race/ethnicity" => "group B", "test preparation course" => "none", "writing grade" => "Pass"}

In [6]:
{training_set, test_set} = Mdl.training_and_test_sets(parsed_dataset, 0.75)

{[%{"gender" => "female", "lunch" => "standard", "parental level of education" => "master's degree", "race/ethnicity" => "group D", "test preparation course" => "none", "writing grade" => "Pass"}, %{"gender" => "female", "lunch" => "standard", "parental level of education" => "high school", "race/ethnicity" => "group D", "test preparation course" => "none", "writing grade" => "Pass"}, %{"gender" => "female", "lunch" => "standard", "parental level of education" => "associate's degree", "race/ethnicity" => "group E", "test preparation course" => "none", "writing grade" => "Pass"}, %{"gender" => "female", "lunch" => "standard", "parental level of education" => "master's degree", "race/ethnicity" => "group E", "test preparation course" => "completed", "writing grade" => "Pass"}, %{"gender" => "male", "lunch" => "standard", "parental level of education" => "high school", "race/ethnicity" => "group D", "test preparation course" => "none", "writing grade" => "Pass"}, %{"gender" => "female", "

In [7]:
discrete_attributes = [
  "gender",
  "lunch",
  "parental level of education",
  "race/ethnicity",
  "test preparation course"
]
class = "writing grade"
f = DT.classifier(training_set, discrete_attributes, class)

#Function<5.94015813/1 in Emel.Ml.DecisionTree.classifier/3>

In [8]:
observations = Enum.map(test_set, fn %{"writing grade" => grade} -> grade end)
Enum.take(observations, 5)

["Pass", "Sub-Pass", "Pass", "Pass", "Pass"]

In [9]:
predictions = Enum.map(test_set, fn row -> f.(row) end)
Enum.take(predictions, 5)

["Pass", "Pass", "Pass", "Fail", "Pass"]

In [10]:
Emel.Math.Statistics.similarity(predictions, observations)

0.9